In [236]:
from __future__ import division
import pickle
import re
import os, os.path
import pandas as pd
import pickle
import sys
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss, confusion_matrix
import io

In [237]:
#output_path='dict_full' #where to store the output
label_csv_path='../trainLabels.csv' #path to csv file mapping filenames to labels
data_path = '../train/' #path to the data

dump_path = '../new_opcodes.txt'
unpickled_file=open(dump_path,'rb')
unpickled_dict=pickle.load(unpickled_file)
raw_opcode_list=unpickled_dict.keys() #list of opcodes to filter
#raw_opcode_list=['mov','add','sub','imul'] #list of opcodes to filter
from sklearn.cross_validation import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression

import numpy as np
import os.path

dt = np.dtype([('Id', 'a30'), ('Class', 'u2')])
data = np.loadtxt(label_csv_path, skiprows=1, delimiter = ',', dtype=dt)

X = np.zeros((data.shape[0], len(raw_opcode_list)))
Y = data['Class']

In [222]:
f = open('../syscount.txt', 'r')
sysCountTrain = []
for line in f:
    sysCountTrain.append([int(line.split(":")[1][:-1])])
#     import numpy as np
# myarray = np.asarray(sysCountTrain)
# vals = np.zeros(len(myarray))
# for i in range(0,len(myarray)):
#     vals[i]=i
# arr = np.zeros((len(myarray),2))

# for i in range(0,len(myarray)):
#     arr[i][0] = vals[i]
#     arr[i][1] = myarray[i]

In [132]:
sysCountTrain

[[285],
 [264],
 [102],
 [14],
 [64],
 [10],
 [237],
 [146],
 [64],
 [212],
 [87],
 [99],
 [37],
 [33],
 [319],
 [81],
 [85],
 [33],
 [38],
 [91],
 [21],
 [141],
 [94],
 [65],
 [964],
 [190],
 [101],
 [34],
 [38],
 [33],
 [23],
 [31],
 [164],
 [33],
 [0],
 [39],
 [37],
 [45],
 [0],
 [134],
 [47],
 [158],
 [155],
 [93],
 [37],
 [74],
 [38],
 [33],
 [91],
 [39],
 [64],
 [148],
 [95],
 [39],
 [93],
 [64],
 [41],
 [101],
 [32],
 [33],
 [95],
 [6],
 [99],
 [53],
 [33],
 [122],
 [38],
 [33],
 [14],
 [72],
 [298],
 [38],
 [33],
 [48],
 [33],
 [106],
 [34],
 [33],
 [203],
 [149],
 [0],
 [418],
 [34],
 [34],
 [207],
 [132],
 [114],
 [104],
 [133],
 [38],
 [78],
 [27],
 [98],
 [40],
 [38],
 [34],
 [78],
 [33],
 [34],
 [64],
 [259],
 [187],
 [99],
 [33],
 [109],
 [32],
 [84],
 [593],
 [150],
 [262],
 [97],
 [140],
 [80],
 [64],
 [16],
 [113],
 [33],
 [125],
 [38],
 [33],
 [282],
 [74],
 [78],
 [185],
 [32],
 [38],
 [107],
 [79],
 [64],
 [428],
 [185],
 [105],
 [93],
 [39],
 [137],
 [67],
 [133],


In [168]:
from sklearn.cluster import KMeans
clf = KMeans(n_clusters=9, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances='auto', verbose=0, random_state=None, copy_x=True, n_jobs=1)
clf.fit(sysCountTrain)
p = clf.predict(sysCountTrain)

In [169]:
numCorrect = 0
for i in range(len(p)):
    if p[i] == Y[i] - 1:
        numCorrect = numCorrect + 1

In [170]:
numCorrect,len(p)

(1276, 10868)

In [134]:
correct=0
total = 0
for i in range(0,len(sysCountTrain)):
    if(p[i]==Y[i]):
        correct+=1
    total+=1

correct/total

0.03413691571586308

In [171]:
labels = np.zeros(9)

for i in range(0,len(sysCountTrain)):
    labels[p[i]-1]+=1
labels

array([  7.00000000e+00,   3.07000000e+02,   2.00000000e+01,
         1.26900000e+03,   4.00000000e+00,   3.80000000e+01,
         1.23000000e+02,   3.19900000e+03,   5.90100000e+03])

In [128]:
clf.get_params(deep=True)

{'copy_x': True,
 'init': 'k-means++',
 'max_iter': 300,
 'n_clusters': 9,
 'n_init': 10,
 'n_jobs': 1,
 'precompute_distances': 'auto',
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

In [172]:
sysCountTrain.shape

AttributeError: 'list' object has no attribute 'shape'

In [238]:
from os import listdir
from os.path import isfile, join

X = np.zeros((data.shape[0], 2))

for i, (Id, Class) in enumerate(data):
    X[i][0] = os.path.getsize('../train/'+Id[1:-1]+'.asm')
    #X[i][1] = os.path.getsize('../train/'+Id[1:-1]+'.bytes')
    X[i][1] = sysCountTrain[i][0]
 
mypath = '../test'
list_files = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
set_files = set([ os.path.splitext(el)[0] for el in list_files ])
 
Id_list = list(set_files) 
 
X_test = np.zeros(( len(set_files), 2 ))
 
print X_test.shape
 
for i, Id in enumerate(Id_list):
    X_test[i][0] = os.path.getsize('../test/'+Id+'.asm')
    X_test[i][1] = os.path.getsize('../test/'+Id+'.bytes')

(10873, 2)


In [224]:
X

array([[  8.19880000e+04,   2.85000000e+02],
       [  4.44952400e+06,   2.64000000e+02],
       [  9.80898000e+05,   1.02000000e+02],
       ..., 
       [  1.28729900e+06,   1.00000000e+00],
       [  2.11692670e+07,   1.00000000e+00],
       [  8.75987600e+06,   1.00000000e+00]])

In [209]:
from sklearn.cluster import KMeans
clf = KMeans(n_clusters=9, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances='auto', verbose=0, random_state=None, copy_x=True, n_jobs=1)
clf.fit(X)
p = clf.predict(X)

In [245]:
from scipy.cluster.vq import kmeans,vq
#from pylab import plot,show,savefig
#data=X
import matplotlib.pyplot as plt
centroids,_ = kmeans(X,9)
idx,_ = vq(X,centroids)

plt.axis((0,1.6*10**7, 0, 2*10**2))
plt.plot(X[idx==0,0],X[idx==0,1],'ob',
      X[idx==1,0],X[idx==1,1],'or',
      X[idx==2,0],X[idx==2,1],'og') # third cluster points
plt.plot(centroids[:,0],centroids[:,1],'sm',markersize=8)
plt.xlabel('ASM File Size')
plt.ylabel('Byte File Size')
plt.title('Class Distribution of file sizes of ASM and Byte files')
#plt.show()
plt.savefig('kmeans.png')

In [208]:
import matplotlib.pyplot as plt

plt.show()

In [210]:
p

array([3, 1, 3, ..., 3, 6, 1], dtype=int32)